### Setup

In [1]:
import os
from pathlib import Path
import yaml
import dotenv
import random
import string

from cognite.client import CogniteClient, ClientConfig
from cognite.client.credentials import OAuthClientCredentials
from cognite.client.data_classes.data_modeling import (
    SpaceApply,
    ContainerApply,
    ContainerProperty,
    NodeApply,
    NodeOrEdgeData,
    data_types,
)
from cognite.client.data_classes.data_modeling.containers import BTreeIndex
from cognite.client.exceptions import CogniteAPIError

dotenv.load_dotenv()

client = CogniteClient(
    ClientConfig(
        client_name="container-state-demo",
        base_url=os.getenv("COGNITE_BASE_URL"),
        project=os.getenv("COGNITE_PROJECT"),
        credentials=OAuthClientCredentials(
            token_url=os.getenv("COGNITE_TOKEN_URL"),
            client_id=os.getenv("COGNITE_CLIENT_ID"),
            client_secret=os.getenv("COGNITE_CLIENT_SECRET"),
            scopes=[os.getenv("COGNITE_TOKEN_SCOPES")],
        ),
    )
)
dms = client.data_modeling

### Create a demo container

In [2]:
space = dms.spaces.apply(SpaceApply(
    space="container-state-demo",
    description="Space for demoing container state fields"
)).space
container = dms.containers.apply(ContainerApply(
    space=space,
    external_id="state_demo",
    properties={
        "text_prop": ContainerProperty(
            type=data_types.Text,
        ),
    }
))

### Add some data

In [3]:
dms.instances.apply([
    NodeApply(space=space, external_id=f"node-{i}", sources=[
            NodeOrEdgeData(
            source=container.as_id(),
            properties={
                "text_prop": str(i),
            }
        )
    ])
    for i in range(10)
])

InstancesApplyResult(nodes=[<NodeApplyResult(space='container-state-demo', external_id='node-0', version=1) at 0x7d8e69d09a60>, <NodeApplyResult(space='container-state-demo', external_id='node-1', version=1) at 0x7d8e69d09a00>, <NodeApplyResult(space='container-state-demo', external_id='node-2', version=1) at 0x7d8e69d09af0>, <NodeApplyResult(space='container-state-demo', external_id='node-3', version=1) at 0x7d8e69d09b20>, <NodeApplyResult(space='container-state-demo', external_id='node-4', version=1) at 0x7d8e69d09b50>, <NodeApplyResult(space='container-state-demo', external_id='node-5', version=1) at 0x7d8e69d09a90>, <NodeApplyResult(space='container-state-demo', external_id='node-6', version=1) at 0x7d8e69d09be0>, <NodeApplyResult(space='container-state-demo', external_id='node-7', version=1) at 0x7d8e69d09b80>, <NodeApplyResult(space='container-state-demo', external_id='node-8', version=1) at 0x7d8e69d09cd0>, <NodeApplyResult(space='container-state-demo', external_id='node-9', ver

### Add an index

In [ ]:
container_apply = container.as_apply()
container_apply.indexes = {"test_index": BTreeIndex(properties=["text_prop"])}
container = dms.containers.apply(container_apply)
container.indexes["test_index"]

The index is still building! Check back later..

In [ ]:
dms.containers.retrieve(container.as_id()).indexes["test_index"]

### Break the index

In [ ]:
try:
    dms.instances.apply([
        NodeApply(space=space, external_id="bad_node_longstr", sources=[
            NodeOrEdgeData(
                source=container.as_id(),
                properties={
                    "text_prop": "".join(random.choice(string.ascii_letters) for i in range(3000)),
                }
            )
        ]),
    ])
except CogniteAPIError as e:
    print(e.message)

In [ ]:
dms.containers.delete_indexes([(container.as_id(), "test_index")])
dms.instances.apply([
    NodeApply(space=space, external_id="bad_node_longstr", sources=[
        NodeOrEdgeData(
            source=container.as_id(),
            properties={
                "text_prop": "".join(random.choice(string.ascii_letters) for i in range(3000)),
            }
        )
    ]),
])
# Re-apply the index
container_apply = dms.containers.retrieve(container.as_id()).as_apply()
container_apply.indexes = {"test_index2": BTreeIndex(properties=["text_prop"])}
container = dms.containers.apply(container_apply)
container.indexes["test_index2"]

In [ ]:
dms.containers.retrieve(container.as_id()).indexes["test_index2"]

### Restrict the property size
We advice setting `max_text_size` on text properties that should be indexed

In [ ]:
container_apply.properties["text_prop"].type.max_text_size = 100
container = dms.containers.apply(container_apply)
container.properties["text_prop"].constraint_state

In [ ]:
dms.containers.retrieve(container.as_id()).properties["text_prop"].constraint_state

In [ ]:
try:
    dms.instances.apply([
        NodeApply(space=space, external_id="bad_node_longstr2", sources=[
            NodeOrEdgeData(
                source=container.as_id(),
                properties={
                    "text_prop": "".join(random.choice(string.ascii_letters) for i in range(3000)),
                }
            )
        ]),
    ])
except CogniteAPIError as e:
    print(e.message)

### Delete bad data

In [ ]:
dms.instances.delete((space, "bad_node_longstr"))

In [ ]:
dms.containers.delete_indexes([(container.as_id(), "test_index")])

The constraint won't be revalidated automatically

In [ ]:
container_apply = dms.containers.retrieve(container.as_id()).as_apply()
container_apply.properties["text_prop"].type.max_text_size = 101
container = dms.containers.apply(container_apply)
container.properties["text_prop"].constraint_state

In [ ]:
dms.containers.retrieve(container.as_id()).properties["text_prop"].constraint_state

### Re-apply the index

In [ ]:
container_apply = dms.containers.retrieve(container.as_id()).as_apply()
container_apply.indexes = {"test_index": BTreeIndex(properties=["text_prop"])}
container = dms.containers.apply(container_apply)
container.indexes["test_index"]

In [ ]:
dms.containers.retrieve(container.as_id()).indexes["test_index"]

## Also: Container-level constraints
Requires constraints and uniqueness constraints

### Cleanup

In [4]:
dms.instances.delete([(space, extid) for extid in [f"node-{i}" for i in range(10)]])
dms.containers.delete((space, "state_demo"))
dms.spaces.delete(space)

['container-state-demo']